In [3]:
import mediapipe as mp
import cv2
import csv
import os
import numpy as np
import pandas as pd
import pickle
import pygame

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from playsound import playsound
import threading

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
mp_drawing = mp.solutions.drawing_utils   #Ayudas para dibujar
mp_holistic = mp.solutions.holistic       #Soluciones de Mediapipe

In [5]:
# Conectamos a la cámara frontal (Webcam)
cap = cv2.VideoCapture(0)

# Iniciamos el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()      
        
        # Reformatear: BGR -> RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Para proteger las imágenes
        image.flags.writeable = False
        
        #Aplicamos el process para obtener el modelo holistico:
        results = holistic.process(image)               
        
        # RGB -> BGR para renderizar
        image.flags.writeable = True  
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Mostramos la imagen                
        cv2.imshow('Raw Webcam Feed', image)

        # Para salir, pulsamos la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [6]:
results.face_landmarks.landmark[0]

x: 0.436918378
y: 0.592820346
z: -0.0411523543

In [7]:
print("x:", results.pose_landmarks.landmark[0].x,"y:", 
      results.pose_landmarks.landmark[0].y,"z:", 
      results.pose_landmarks.landmark[0].z)

x: 0.39321574568748474 y: 0.500206708908081 z: -0.9265204668045044


In [8]:
#Tenemos 501 coordenadas en total
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [9]:
#la primera columna servirá de identificación de clase 
landmarks = ['class']

for val in range(1,num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

#Mostramos ejemplo:
print(landmarks[0:13])

['class', 'x1', 'y1', 'z1', 'v1', 'x2', 'y2', 'z2', 'v2', 'x3', 'y3', 'z3', 'v3']


In [10]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [11]:
class_name = "Cansado"

In [12]:
#Conectamos a la cámara frontal (Webcam)
cap = cv2.VideoCapture(0)

# Iniciamos el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()
        
        # #Reformatear: BGR -> RGR
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Para proteger las imágenes
        image.flags.writeable = False
        
        #Aplicamos el process para obtener el holistico
        results = holistic.process(image)

        # RGB -> BGR para renderizar
        image.flags.writeable = True  
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
  
        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Exportamos las coordenadas
        try:

            # Guardamos los datos del pose y face en un np array
            # Extraemos Pose landmarks ( el método .flatten() convierte el np array en 1D)
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in pose]).flatten())
           
            # Extraemos Face landmarks            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in face]).flatten())
        
            row = pose_row + face_row
            row.insert(0,class_name)
            
            #Exportar a CSV
            with open('coords.csv',mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass
        
        #Mostramos la imagen
        cv2.imshow('Raw Webcam Feed', image)

        # Para salir, pulsamos la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [13]:
class_name = "Despierto"

In [14]:
#Conectamos a la cámara frontal (Webcam)
cap = cv2.VideoCapture(0)

# Iniciamos el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()
        
        # #Reformatear: BGR -> RGR
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Para proteger las imágenes
        image.flags.writeable = False
        
        #Aplicamos el process para obtener el holistico
        results = holistic.process(image)

        # RGB -> BGR para renderizar
        image.flags.writeable = True  
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
  
        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Exportamos las coordenadas
        try:

            # Guardamos los datos del pose y face en un np array
            # Extraemos Pose landmarks ( el método .flatten() convierte el np array en 1D)
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in pose]).flatten())
           
            # Extraemos Face landmarks            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]
                                      for landmark in face]).flatten())
        
            row = pose_row + face_row
            row.insert(0,class_name)
            
            #Exportar a CSV
            with open('coords.csv',mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass
        
        #Mostramos la imagen
        cv2.imshow('Raw Webcam Feed', image)

        # Para salir, pulsamos la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [15]:
# Leemos el archivo .csv
df = pd.read_csv("coords.csv")
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Cansado,0.421593,0.512339,-0.988697,0.999841,0.460478,0.445553,-0.937463,0.999553,0.483096,...,-0.005061,0.0,0.542365,0.437736,0.022243,0.0,0.548907,0.430032,0.022667,0.0
1,Cansado,0.421760,0.511946,-0.964806,0.999832,0.460946,0.445273,-0.914203,0.999540,0.483610,...,-0.005440,0.0,0.542348,0.444873,0.022031,0.0,0.548599,0.436489,0.022582,0.0
2,Cansado,0.422060,0.511534,-0.921107,0.999825,0.461176,0.444854,-0.871662,0.999529,0.483761,...,-0.005229,0.0,0.541631,0.449978,0.020740,0.0,0.548070,0.441174,0.021375,0.0
3,Cansado,0.421957,0.510798,-0.872436,0.999808,0.461131,0.444304,-0.828644,0.999494,0.483744,...,-0.006526,0.0,0.539031,0.454149,0.016890,0.0,0.545646,0.445335,0.017389,0.0
4,Cansado,0.423545,0.521515,-0.896618,0.999798,0.464074,0.452020,-0.850341,0.999473,0.486716,...,-0.009371,0.0,0.537203,0.466181,0.013919,0.0,0.543504,0.457318,0.014379,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,Despierto,0.402532,0.537713,-1.350729,0.999696,0.453193,0.453025,-1.289653,0.999177,0.484212,...,-0.005687,0.0,0.551718,0.461248,0.018816,0.0,0.558665,0.452720,0.019254,0.0
860,Despierto,0.402691,0.534990,-1.344067,0.999693,0.453619,0.450626,-1.286946,0.999170,0.484629,...,-0.003995,0.0,0.547475,0.462675,0.020136,0.0,0.554586,0.453009,0.020762,0.0
861,Despierto,0.400029,0.534834,-1.337268,0.999709,0.452630,0.450125,-1.277541,0.999211,0.484028,...,-0.006928,0.0,0.546755,0.457086,0.013895,0.0,0.553856,0.447550,0.014105,0.0
862,Despierto,0.406802,0.535893,-1.335828,0.999716,0.456913,0.451669,-1.272244,0.999229,0.487036,...,-0.006784,0.0,0.546588,0.457224,0.017803,0.0,0.553657,0.447426,0.018287,0.0


In [16]:
# Guardaremos las variables en X y el target en Y
X = df.drop('class',axis=1)
y = df['class']

In [17]:
# Dividimos el dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1000)

In [18]:
# Definimos los cuatro pipelines
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [19]:
# Guardamos los modelos en la biblioteca fit_models
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [20]:
# Probamos a predecir X_test con el modelo de Random Forest
fit_models['rf'].predict(X_test)

array(['Despierto', 'Despierto', 'Cansado', 'Cansado', 'Despierto',
       'Cansado', 'Despierto', 'Despierto', 'Despierto', 'Despierto',
       'Cansado', 'Despierto', 'Cansado', 'Cansado', 'Despierto',
       'Cansado', 'Cansado', 'Despierto', 'Despierto', 'Cansado',
       'Despierto', 'Cansado', 'Despierto', 'Cansado', 'Cansado',
       'Cansado', 'Cansado', 'Cansado', 'Cansado', 'Cansado', 'Despierto',
       'Despierto', 'Cansado', 'Cansado', 'Despierto', 'Despierto',
       'Despierto', 'Cansado', 'Despierto', 'Despierto', 'Cansado',
       'Despierto', 'Despierto', 'Despierto', 'Cansado', 'Despierto',
       'Cansado', 'Despierto', 'Cansado', 'Cansado', 'Cansado', 'Cansado',
       'Despierto', 'Despierto', 'Despierto', 'Despierto', 'Cansado',
       'Cansado', 'Cansado', 'Cansado', 'Despierto', 'Despierto',
       'Despierto', 'Cansado', 'Despierto', 'Cansado', 'Despierto',
       'Cansado', 'Cansado', 'Despierto', 'Cansado', 'Despierto',
       'Cansado', 'Despierto', 'Despie

In [21]:
#Predecir las clases de X_test
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    
    #Mostramos las precisiones de cada modelo
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 0.9961538461538462
gb 0.9961538461538462


In [23]:
# Escribimos el modelo en un archivo .pkl
with open('cansado_despierto.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [24]:
# Abrimos de nuevo el modelo
with open('cansado_despierto.pkl', 'rb') as f:
    model = pickle.load(f)

In [25]:
# Inicializar pygame mixer
pygame.mixer.init()
alarm_sound = pygame.mixer.Sound('alarma.wav')  # Asegúrate de que el archivo de sonido esté en formato .wav

In [26]:
# Conectamos la cámara
cap = cv2.VideoCapture(0)

# Inicializar la variable de control de la alarma
alarm_playing = False

# Inicia el modelo holístico
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():

        # Se guarda la imagen en frame
        ret, frame = cap.read()

        # Verificar que la imagen se haya leído correctamente
        if not ret:
            print("No se pudo recibir imagen de la cámara. Saliendo...")
            break

        # BGR -> RGB y seguridad de la imagen
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Detecciones
        results = holistic.process(image)

        # RGB -> BGR y seguridad de la imagen
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Dibujo de landmarks
        # 1. Face landmarks
        mp_drawing.draw_landmarks(
            image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        )

        # 2. Right hand
        mp_drawing.draw_landmarks(
            image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        )

        # 3. Left Hand
        mp_drawing.draw_landmarks(
            image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
        )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )

        # Exporta coordenadas
        try:
            # Verificar que se hayan detectado landmarks de pose y rostro
            if results.pose_landmarks and results.face_landmarks:
                # Extraemos Pose landmarks
                pose = results.pose_landmarks.landmark
                pose_row = list(np.array([
                    [landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose
                ]).flatten())

                # Extraemos Face landmarks
                face = results.face_landmarks.landmark
                face_row = list(np.array([
                    [landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face
                ]).flatten())

                # Concatenamos las filas
                row = pose_row + face_row

                # Hacemos detecciones
                X = pd.DataFrame([row])

                # Predecir la clase
                body_language_class = model.predict(X)[0]

                # Predecir la probabilidad de la clase
                body_language_prob = model.predict_proba(X)[0]

                # Calcular la probabilidad
                probability = round(body_language_prob[np.argmax(body_language_prob)], 2)

                # Mostrar valores para depuración
                print(f"Clase predicha: '{body_language_class}'")
                print(f"Probabilidad: {probability}")

                # Recogemos coordenadas de la oreja izquierda para visualizar texto dinámicamente
                coords = tuple(np.multiply(
                    np.array(
                        (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)
                    ), [640,480]).astype(int))

                # Mostramos la clase en la oreja izquierda
                cv2.rectangle(
                    image,
                    (coords[0], coords[1]+5),
                    (coords[0]+len(body_language_class)*20, coords[1]-30),
                    (245, 117, 16), -1)
                cv2.putText(
                    image, body_language_class, coords,
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Status Box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

                # Mostramos Class
                cv2.putText(
                    image, 'CLASS', (95,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(
                    image, body_language_class.split(' ')[0], (90,40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Mostramos Probabilidad
                cv2.putText(
                    image, 'PROB', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(
                    image, str(probability), (10,40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Al estar muy cansado, activamos la alerta de descanso y la alarma
                if (probability > 0.70) and ("Cansado" in body_language_class):

                    # Mostrar mensaje de alerta
                    cv2.rectangle(
                        image,
                        (coords[0], coords[1]+5),
                        (coords[0]+len('MUY CANSADO')*20, coords[1]-30),
                        (0, 0, 255), -1)

                    cv2.putText(
                        image, 'MUY CANSADO', coords,
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                    cv2.rectangle(image, (0,0), (550, 60), (0, 0, 255), -1)

                    cv2.putText(
                        image, "ALERTA! NECESITAS DESCANSAR", (15,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                    # Activar alarma si no está sonando
                    if not alarm_playing:
                        alarm_playing = True
                        alarm_sound.play(-1)  # Reproducir en bucle
                else:
                    if alarm_playing:
                        alarm_playing = False
                        alarm_sound.stop()
            else:
                print("No se detectaron landmarks de pose o rostro.")
                # Puedes manejar este caso si es necesario

        except Exception as e:
            print(f"Ocurrió un error en el procesamiento: {e}")
            pass

        # Mostrar la imagen
        cv2.imshow('DETECTOR DE SOMNOLENCIA', image)

        # Salir con la tecla 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Liberar la cámara y cerrar ventanas
    cap.release()
    cv2.destroyAllWindows()
    pygame.quit()

c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Clase predicha: 'Despierto'
Probabilidad: 0.82


c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Clase predicha: 'Despierto'
Probabilidad: 0.84


c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

Clase predicha: 'Despierto'
Probabilidad: 0.59


c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Clase predicha: 'Despierto'
Probabilidad: 0.62


c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Leo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Clase predicha: 'Despierto'
Probabilidad: 0.77
